<a href="https://colab.research.google.com/github/azimhims/azimhims/blob/main/Pretrain_Assignment_Utkface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# need to upload datasets from kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [2]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
100% 331M/331M [00:10<00:00, 32.1MB/s]
100% 331M/331M [00:10<00:00, 34.4MB/s]


In [3]:
import zipfile

In [27]:
with zipfile.ZipFile('/content/utkface-new.zip','r') as zip_ref:
  zip_ref.extractall('/content')

In [28]:
import os
import keras
import tensorflow
import numpy as np
import pandas as pd
#from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator





In [29]:
tensorflow.config.run_functions_eagerly(True)

In [30]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'
folder = '/content/utkface_aligned_cropped/'

In [31]:
data = os.listdir(folder_path)

In [32]:
print(data[0])

3_1_0_20170109192805549.jpg.chip.jpg


In [33]:
age = []
gender = []
race = []
img_path = []
for i in data:
  age.append(i.split('_')[0])
  gender.append(i.split('_')[1])
  race.append(i.split('_')[2])
  img_path.append(i)

In [34]:
img_path[0]

'3_1_0_20170109192805549.jpg.chip.jpg'

In [35]:
print(len(age),len(gender),len(race),len(img_path))

23708 23708 23708 23708


In [36]:
# need to add in dataframe
df = pd.DataFrame({'age':age,'gender':gender,'race':race,'img_path':img_path})

In [37]:
df

,age,gender,race,img_path
0,3,1,0,3_1_0_20170109192805549.jpg.chip.jpg
1,45,0,0,45_0_0_20170117191748012.jpg.chip.jpg
2,38,1,1,38_1_1_20170117142941700.jpg.chip.jpg
3,26,1,2,26_1_2_20170117154930236.jpg.chip.jpg
4,4,0,0,4_0_0_20170104010859393.jpg.chip.jpg
...,...,...,...,...
23703,16,1,3,16_1_3_20170104223028462.jpg.chip.jpg
23704,28,0,0,28_0_0_20170120221648261.jpg.chip.jpg
23705,39,1,1,39_1_1_20170116021937896.jpg.chip.jpg
23706,35,0,4,35_0_4_20170104201734834.jpg.chip.jpg


In [14]:
!pip install split-folders

In [38]:
# train_data = df.sample(frac=1,random_state=0).iloc[:20000]
# test_data = df.sample(frac=1,random_state= 0).iloc[20000:]
#this is alternative option for update the train and test foler with the help of splitfolders
import splitfolders
splitfolders.ratio(folder, output="data", seed=1337, ratio=(.8, 0.1,0.1))

Copying files: 33488 files [00:06, 5450.22 files/s]


In [16]:
#print(train_data.shape, test_data.shape)

NameError: name 'train_data' is not defined

In [39]:
train_data = ImageDataGenerator(rescale=1./255,
                                horizontal_flip = True,
                                rotation_range= .10,
                                zoom_range = .20,

                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2
                                  )
test_data = ImageDataGenerator(rescale=1./255)


In [50]:
train_dir = '/content/data/train/'
train_gen = train_data.flow_from_directory(directory = train_dir,
                                           target_size= (200,200),
                                           batch_size= 16,
                                           class_mode = 'sparse'
                                           )

test_dir = '/content/data/test/'
test_gen = test_data.flow_from_directory(directory = test_dir,
                                           target_size= (200,200),
                                           batch_size= 16,
                                           class_mode = 'sparse'
                                           )
valid_dir = '/content/data/val/'
valid_gen = test_data.flow_from_directory(directory = valid_dir,
                                           target_size= (200,200),
                                           batch_size= 16,
                                          class_mode = 'sparse'
                                        )

Found 26790 images belonging to 2 classes.
Found 3350 images belonging to 2 classes.
Found 3348 images belonging to 2 classes.


In [41]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [51]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

In [43]:
resnet.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 262, 262, 3)    │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 128, 128, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 128, 128, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 128, 128, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 130, 130, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 64, 64, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 64, 64, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 64, 64, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 64, 64, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 64, 64, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 64, 64, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 64, 64, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 64, 64, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 64, 64, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
tst = resnet.layers[-1].output

In [70]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(128, activation='relu')(flatten)
doc_controls = Dropout(0.5)(dense1)
dense2 = Dense(128,activation='relu')(flatten)
doc_controls2 = Dropout(0.5)(dense2)

dense3 = Dense(128,activation='relu')(flatten)
doc_controls3 = Dropout(0.5)(dense3)
# dense4 = Dense(128,activation='relu')(dense1)
# dense5 = Dense(128,activation='relu')(dense2)
# dense6 = Dense(128,activation='relu')(dense3)


output1 = Dense(1,activation = 'softmax',name='race')(doc_controls)
output2 = Dense(1,activation='sigmoid',name='gender')(doc_controls2)

output3 = Dense(1,activation='linear',name='age')(doc_controls3)

model = Model(inputs=resnet.input, outputs=[output1,output2,output3])

In [71]:
model.compile(optimizer='adam', loss={'race':'categorical_crossentropy','age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'race':'accuracy','age': 'mae', 'gender': 'accuracy'})
#,loss_weights={'age':99.0,'gender':1.0,'race':5.0})


In [ ]:
import tensorflow as tf

tf.config.run_functions_eagerly(True)


In [ ]:
model.fit(train_gen, batch_size=16, epochs=5, validation_data=test_gen,verbose=1)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (16, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/losses/losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(16, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **se

   1/1675 ━━━━━━━━━━━━━━━━━━━━ 45:08 2s/step - loss: 0.0000e+00 - race_accuracy: 0.1875

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


1031/1675 ━━━━━━━━━━━━━━━━━━━━ 6:01 561ms/step - loss: 0.0000e+00 - race_accuracy: 0.2942

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (6, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/losses/losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(6, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


1675/1675 ━━━━━━━━━━━━━━━━━━━━ 1014s 605ms/step - loss: 0.0000e+00 - race_accuracy: 0.2935 - val_loss: 0.0000e+00 - val_race_accuracy: 0.2919
Epoch 2/5
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 988s 573ms/step - loss: 0.0000e+00 - race_accuracy: 0.2929 - val_loss: 0.0000e+00 - val_race_accuracy: 0.2919
Epoch 3/5
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 970s 566ms/step - loss: 0.0000e+00 - race_accuracy: 0.2927 - val_loss: 0.0000e+00 - val_race_accuracy: 0.2919
Epoch 4/5
 359/1675 ━━━━━━━━━━━━━━━━━━━━ 11:42 534ms/step - loss: 0.0000e+00 - race_accuracy: 0.2883